# Hard and soft constraints

### Hard constraints
These are constraints that must be pursued during the search and without meeting these constraints, the solution is not true. These constraints are:
- One teacher cannot give two courses at the same time
- A set of students cannot have two classes at the same time
- A teacher cannot teach more than maximum allotted hours.
- Classes Cannot be on off days (Saturday and Sunday)
- Maximum number of hours allotted for a day cannot be exceeded.

### Soft constraints
These constraints that are not mandatory for a solution must be pursued, but the quality of the courses schedule is decided by following these constraints
- In the timetable of teacher, no more than two lectures a day, each one 1 hours and 30 minutes, breaks are preferred 30 minutes or more.
- In the students' timetable, we have to put three lectures a day at maximum with 30 minutes breaks between lectures so that they have more time for self-study at home
- We must ensure that a group of students does not have to attend school only to attend one class, there has to be a maximum of three classes in a day and a minimum of two classes so that it will be worth it to come to college on a particular day

# Initialization

In [879]:
def initialize(students, courses, day_times, prof_input):
    initialized_data = {}
    initialized_data["stgs"] = students
    initialized_data["courses"] = courses
    initialized_data["days"] = [i for i in range(day_times["day"])]
    initialized_data["periods"] = [i for i in range(day_times["period"])]
    initialized_data["profs"] = prof_input
    return initialized_data

In [880]:
# Fitness function
def fitness_function(matrix, data):
    stgs = [ key for key in data["stgs"].keys()]
    courses = [ key for key in data["courses"].keys()]
    profs = [key for key in data["profs"].keys()]
    stg_checker = {}
    course_checker = {}
    prof_checker = {}
    for stg in stgs:
        stg_checker[stg] = []
    for prof in profs:
        prof_checker[prof] = []
    for course in courses:
        course_checker[course] = 0

    hard_penalty_counter = 0
    for chromosome in matrix:
        stg = chromosome["stg"]
        course = chromosome["course"]
        prof = chromosome["prof"]
        day = chromosome["day"]
        period = chromosome["period"]
        course_checker[course] += 1
        if course not in data["stgs"][stg]["course_list"]:
            hard_penalty_counter += 1
        if prof not in data["courses"][course]["prof_list"]:
            hard_penalty_counter += 1
        timeslot = [day, period]
        if timeslot not in stg_checker[stg]:
            stg_checker[stg].append(timeslot)
        else:
            hard_penalty_counter += 1
        if timeslot not in prof_checker[prof]:
            prof_checker[prof].append(timeslot)
        else:
            hard_penalty_counter += 1
        if timeslot not in data["profs"][prof]["availability_list"]:
            hard_penalty_counter += 1
    for course_key in course_checker.keys():
        if course_checker[course_key] != data["courses"][course_key]["hour"]:
            hard_penalty_counter += 1
    for stg in stgs:
        if len(stg_checker[stg]) != data["stgs"][stg]["hour"]:
            hard_penalty_counter += 1
    
    # Check for soft constraints
    soft_penalty_counter = 0

    result = 1/(1+(soft_penalty_counter + hard_penalty_counter))
    return result

In [891]:
# Code for initialization based on input
# Number of population = number student gouweekly hours
student_input = {0: {"hour": 4, "course_list": [0]}, 1: {"hour": 3, "course_list": [1]}}
course_input = {0: {"hour": 4, "prof_list": [0, 1]}, 1: {"hour": 3, "prof_list": [0, 1]}}
day_time_availability = {"day": 5, "period": 8}
# [[day, period]]
prof_input = {0: {"availability_list": [[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7],[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7],[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7], [3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7],[4,0], [4,1], [4,2], [4,3], [4,4], [4,5], [4,6], [4,7]] },
1: {"availability_list": [[0,0], [0,1], [0,2], [0,3], [0,4], [0,5], [0,6], [0,7],[1,0], [1,1], [1,2], [1,3], [1,4], [1,5], [1,6], [1,7],[2,0], [2,1], [2,2], [2,3], [2,4], [2,5], [2,6], [2,7], [3,0], [3,1], [3,2], [3,3], [3,4], [3,5], [3,6], [3,7]]}}
initialized_data = initialize(student_input, course_input, day_time_availability, prof_input)
print(initialized_data)

{'stgs': {0: {'hour': 4, 'course_list': [0]}, 1: {'hour': 3, 'course_list': [1]}}, 'courses': {0: {'hour': 4, 'prof_list': [0, 1]}, 1: {'hour': 3, 'prof_list': [0, 1]}}, 'days': [0, 1, 2, 3, 4], 'periods': [0, 1, 2, 3, 4, 5, 6, 7], 'profs': {0: {'availability_list': [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [4, 7]]}, 1: {'availability_list': [[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7]]}}}


In [882]:
stgs = [ key for key in initialized_data["stgs"].keys()]
courses = [ key for key in initialized_data["courses"].keys()]
profs = [key for key in initialized_data["profs"].keys()]
days = initialized_data["days"]
periods = initialized_data["periods"]

In [883]:
# Initial individual with fitness value = 1
# one of possible solutions: 
chromosome_0 = {"stg": stgs[0], "course": courses[0], "prof": profs[0], "day": days[0], "period": periods[0]}
chromosome_1 = {"stg": stgs[0], "course": courses[0], "prof": profs[0], "day": days[0], "period": periods[1]}
chromosome_2 = {"stg": stgs[0], "course": courses[0], "prof": profs[0], "day": days[0], "period": periods[2]}
chromosome_3 = {"stg": stgs[0], "course": courses[0], "prof": profs[0], "day": days[0], "period": periods[3]}
chromosome_4 = {"stg": stgs[1], "course": courses[1], "prof": profs[1], "day": days[3], "period": periods[0]}
chromosome_5 = {"stg": stgs[1], "course": courses[1], "prof": profs[1], "day": days[3], "period": periods[1]}
chromosome_6 = {"stg": stgs[1], "course": courses[1], "prof": profs[1], "day": days[3], "period": periods[2]}
individual_0 = [chromosome_0, chromosome_1, chromosome_2, chromosome_3, chromosome_4, chromosome_5, chromosome_6]
print(fitness_function(individual_0, initialized_data))

1.0


In [884]:
import numpy as np
from numpy import random

# Automating population generation
def generate_population(n, input_data):
    stgs = [ key for key in input_data["stgs"].keys()]
    courses = [ key for key in input_data["courses"].keys()]
    profs = [key for key in input_data["profs"].keys()]
    days = input_data["days"]
    periods = input_data["periods"]
    chromosome_length = 0
    for stg in stgs:
        chromosome_length += input_data["stgs"][stg]["hour"]

    population = []
    for _ in range(n):
        cur_individual = []
        for _ in range(chromosome_length):
            stg = random.randint(len(stgs))
            course = random.randint(len(courses))
            prof = random.randint(len(profs))
            day = random.randint(len(days))
            period = random.randint(len(periods))
            cur_chromosome = {"stg": stg, "course": course, "prof": prof,
            "day": day, "period": period}
            cur_individual.append(cur_chromosome)
        population.append(cur_individual)
    return population

In [885]:
# Selection algorithm tournament selection, uniform probability
def tournament_selection(population_fit):
    candidate_1 = random.randint(len(population_fit))
    candidate_2 = random.randint(len(population_fit))
    while candidate_1 == candidate_2:
        candidate_2 = random.randint(len(population_fit))
    if population_fit[candidate_1] > population_fit[candidate_2]:
        return candidate_1
    else:
        return candidate_2

In [886]:
# #One point crossover from index 1 to last index
def one_point_crossover(parent_1, parent_2, crossover_rate=0.8):
    crossover_point = random.randint(1, len(parent_1))
    child_1 = parent_1.copy()
    if (random.random() < crossover_rate):
        child_1[crossover_point:] = parent_2[crossover_point:]
    return child_1

In [887]:
# Mutation by Random resetting
def mutation(individual, input_data, mutation_rate=0.02):
    stgs = [ key for key in input_data["stgs"].keys()]
    courses = [ key for key in input_data["courses"].keys()]
    profs = [key for key in input_data["profs"].keys()]
    days = input_data["days"]
    periods = input_data["periods"]
    # mutate_index = random.randint(len(individual))
    for mutate_index in range(len(individual)):
        mutation_prob = random.random()
        if (mutation_prob < mutation_rate):
            mutate_attr = random.randint(len(individual))
            if mutate_attr == 0:
                individual[mutate_index]["stg"] = random.randint(len(stgs))
            if mutate_attr == 1:
                individual[mutate_index]["course"] = random.randint(len(courses))
            if mutate_attr == 2:
                individual[mutate_index]["prof"] = random.randint(len(profs))
            if mutate_attr == 3:
                individual[mutate_index]["day"] = random.randint(len(days))
            if mutate_attr == 4:
                individual[mutate_index]["period"] = random.randint(len(periods))
    return individual

In [888]:
# selection by ranking
# choose top 0.1 and then reproduce until same length
def get_new_population_selection_by_ranking(population, fitness_values, data):
    zipped_population = [(x,y) for x,y in zip(population, fitness_values)]
    sorted_population = sorted(zipped_population,key=lambda x: x[1], reverse=True)
    population_length = len(population)
    elites = sorted_population[:population_length//10]
    elites = [x[0] for x in elites]
    new_gen = []
    for _ in range(population_length):
        parent_x_idx = random.randint(len(elites))
        parent_y_idx = random.randint(len(elites))
        while parent_x_idx == parent_y_idx:
            parent_y_idx = random.randint(len(elites))
        parent_x, parent_y = elites[parent_x_idx], elites[parent_y_idx]
        child_1= one_point_crossover(parent_x, parent_y)
        mutated_1= mutation(child_1, data)
        new_gen.append(mutated_1)
    return new_gen

In [892]:
population_num = 100
initial_population = generate_population(population_num, initialized_data)
# print(population)
initial_fitness_values = []
for idx in range(len(initial_population)):
    fitness_value = fitness_function(initial_population[idx], initialized_data)
    initial_fitness_values.append(fitness_value)
print(np.max(initial_fitness_values), np.mean(initial_fitness_values))

0.3333333333333333 0.12512684537684537


In [903]:
counter = 0
current_fitness_values = initial_fitness_values.copy()
current_population = initial_population.copy()
while True:
    # if counter == 100:
    #     break
    new_population = get_new_population_selection_by_ranking(current_population, current_fitness_values, initialized_data)
    new_fitness_values= []
    for idx in range(len(new_population)):
        fitness_value = fitness_function(new_population[idx], initialized_data)
        new_fitness_values.append(fitness_value)
    for i in range(len(new_fitness_values)):
        if new_fitness_values[i] > current_fitness_values[i]:
            # print("Switched", i)
            current_population[i] = new_population[i]
            current_fitness_values[i] = new_fitness_values[i]
    counter += 1
    if 1 in new_fitness_values:
        index = new_fitness_values.index(1)
        print("SOLUTION!", counter, new_population[index])
        break
    # if np.mean(new_fitness_values) > np.mean(current_fitness_values):
    #     current_population = new_population
    #     current_fitness_values = new_fitness_values
    if counter % 100 == 0:
        print(np.max(current_fitness_values), np.mean(current_fitness_values))
        # break

0.5 0.32266666666666666
0.5 0.33500000000000013
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.48333333333333334
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
SOLUTION! 2903 [{'stg': 0, 'course': 0, 'prof': 0, 'day': 4, 'period': 0}, {'stg': 0, 'course': 0, 'prof': 0, 'day': 1, 'period': 7}, {'stg': 1, 'course': 1, 'prof': 0, 'day': 2, 'period': 7}, {'stg': 0, 'course': 0, 'prof': 0, 'day': 4, 'period': 2}, {'stg': 0, 'course': 0, 'prof': 1, 'day': 2, 'period': 6}, {'stg': 1, 'course': 1, 'prof': 1, 'day': 2, 'period': 2}, {'stg': 1, 'course': 1, 'prof': 1, 'day': 3, 'period': 5}]
